In [1]:
using Laplacians

### first gaus siedel and jacobi - these seem to be pretty unefficient

In [2]:
include("../../../src/preconditioners.jl")

gs (generic function with 1 method)

In [3]:
n = 10000
a = chimera(n,1);
la = lap(a)
b = rand(n); b = b - mean(b);

In [4]:
@time f = gs(la);

  0.032548 seconds (18.99 k allocations: 999.666 KB)


In [5]:
@time x = pcg(la, b, f, tol = 1e-2, maxits = 10000, verbose=true);

PCG stopped after: 565 iterations with relative error 0.009871433445591653.
  1.395195 seconds (11.82 M allocations: 496.713 MB, 5.93% gc time)


In [6]:
norm(la * x - b) / norm(b)

0.009871433445617637

In [7]:
@time f = jacobi(la);

  0.003035 seconds (525 allocations: 109.604 KB)


In [8]:
@time x = pcg(la, b, f, tol = 1e-2, maxits = 10000, verbose=true);

PCG stopped after: 1176 iterations with relative error 0.009977530963268663.
  0.798011 seconds (53.18 k allocations: 450.139 MB, 8.77% gc time)


In [9]:
norm(la * x - b) / norm(b)

0.0099775309632348

### let's take a look at the algebraic multigrid solver

In [10]:
using PyAMG

In [11]:
@time x = PyAMG.solve(la, b, tol = 1e-7, maxiter = 100);

  2.902239 seconds (1.09 M allocations: 46.708 MB, 0.53% gc time)
  Detected a Hermitian matrix
    maxiter = 100
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
    iteration 10
    iteration 11
    iteration 12
  Residual reduction ||r_k||/||r_0|| = 3.79e-07


/Users/serbanstan/.julia/v0.4/Conda/deps/usr/lib/python2.7/site-packages/pyamg/strength.py:533: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if B == 'ones':
/Users/serbanstan/.julia/v0.4/Conda/deps/usr/lib/python2.7/site-packages/pyamg/krylov/_cg.py:156: UserWarning: 
Indefinite preconditioner detected in CG, aborting

  warn("\nIndefinite preconditioner detected in CG, aborting\n")


In [12]:
norm(la * x - b) / norm(b)

1.163833154476479e-6

In [13]:
?PyAMG.solve

`solve(amg::AMGSolver, b, kwargs...)`

Returns a `Vector` with the result of the AMG solver.

### `kwargs`  (copy-pasted from Python docs)

  * `x0` : Initial guess.
  * `tol` : Stopping criteria: relative residual r[k]/r[0] tolerance.
  * `maxiter` : Stopping criteria: maximum number of allowable iterations.
  * `cycle` : {"V","W","F","AMLI"}     Type of multigrid cycle to perform in each iteration.
  * `accel` : Defines acceleration method.  Can be a string such as "cg"     or "gmres" which is the name of an iterative solver in     `pyamg.krylov` (preferred) or scipy.sparse.linalg.isolve.     If accel is not a string, it will be treated like a function     with the same interface provided by the iterative solvers in SciPy.         (the function version is not tested in Julia!)
  * `callback` : User-defined function called after each iteration.  It is     called as callback(xk) where xk is the k-th iterate vector.
  * `residuals` : List to contain residual norms at each iteration.

`solve(A::SparseMatrixCSC, b::Vector; kwargs...)`:

PyAMG's 'blackbox' solver. See `pyamg.solve?` for `kwargs`.


### now, let's test on harder graphs

In [14]:
a = readFromFile("3.txt");

In [15]:
n = a.n

90000

In [16]:
la = lap(a);
b = rand(n); b = b - mean(b);

In [17]:
@time x = PyAMG.solve(la, b, tol = 1e-6, maxiter = 200);

 12.444722 seconds (221 allocations: 8.230 MB, 0.04% gc time)
  Detected a Hermitian matrix
    maxiter = 200
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
    iteration 10
    iteration 11
    iteration 12
    iteration 13
    iteration 14
    iteration 15
    iteration 16
    iteration 17
    iteration 18
    iteration 19
    iteration 20
    iteration 21
    iteration 22
    iteration 23
    iteration 24
    iteration 25
    iteration 26
    iteration 27
    iteration 28
    iteration 29
    iteration 30
    iteration 31
    iteration 32
    iteration 33
    iteration 34
    iteration 35
    iteration 36
    iteration 37
    iteration 38
    iteration 39
    iteration 40
    iteration 41
    iteration 42
    iteration 43
    iteration 44
    iteration 45
    iteration 46
    iteration 47
    iteration 48
    iteration 49
    iteration 50
    iteration 51
    iteration 52
    iteration 5

In [18]:
norm(la * x - b) / norm(b)

814409.1202502114

### now, our solver

In [19]:
include("../../../src/samplingTreeSolver.jl");

In [20]:
@time f = samplingSolver(a, tol = 1e-6, maxits = 1000, eps = 0.5, sampConst = 0.02, beta = 1000.0);

rho = 10.41060304582247
Average number of multiedges = 5.2266946715106295
Maximum number of multiedges = 10.410603085816941
Time to build the tree and compute the stretch: elapsed time: 1.154066884 seconds
  5.050166 seconds (19.78 M allocations: 1.002 GB, 7.35% gc time)


In [21]:
@time norm(la * f(b) - b) / norm(b)

  1.469140 seconds (23.99 M allocations: 920.352 MB, 9.56% gc time)


9.416582523407183e-7

In [23]:
la = lap(a);

In [24]:
f = gs(la);

In [30]:
@time x = pcg(la, b, f, tol = 1e-2, maxits = 10000, maxtime=20, verbose=true);

PCG stopped at maxtime.
PCG stopped after: 1449 iterations with relative error 1.203890738292768.
 20.009877 seconds (261.07 M allocations: 10.698 GB, 7.83% gc time)


In [31]:
norm(la * x - b) / norm(b)

1.2038907382925188